In [1]:
import os
import pandas as pd
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import MapType, StringType, IntegerType, StructType, StructField, FloatType, ArrayType

spark = SparkSession.builder.appName("").getOrCreate()
spark.conf.set('spark.sql.session.timeZone', 'UTC')

In [3]:
path_to_data = '/user/spf248/twitter/data'

In [ ]:
users = spark.read.json(os.path.join(path_to_data,'users','users-profile'))
locations = spark.read.option('header','true').option('multiLine','true').csv(os.path.join(path_to_data,'locations','profiles','account-locations.csv'))

In [ ]:
users = users.withColumnRenamed('location','user_location')
users = users.withColumnRenamed('id_str','user_id')
users = users.join(locations.select('user_location','country_short'),on='user_location')
users = users.withColumnRenamed('country_short','country_code')
users = users.withColumn('created_at', F.to_timestamp('created_at',"EEE MMM dd HH:mm:ss ZZZZZ yyyy"))

In [ ]:
users.write.partitionBy("country_code").mode("overwrite").format("orc").save(os.path.join(path_to_data,'users','by-country'))